In [11]:
from ultralytics import YOLO
import numpy as np
import supervision as sv
import time


In [ ]:
# Path setup
video_path = "C:/Users/Jatin/SigProj/Signal.mp4"
output_path = "C:/Users/Jatin/SigProj/signal-result.mp4"

# Polygon zone
polygon = np.array([
    [220, 80],
    [0, 200], 
    [0, 400],
    [513, 400],
    [465, 80]
])

# Video info
video_info = sv.VideoInfo.from_video_path(video_path)
zone = sv.PolygonZone(polygon=polygon)

# Annotators
box_annotator = sv.BoundingBoxAnnotator(thickness=2)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color.WHITE, thickness=1, text_thickness=1, text_scale=1)

# Track last count
last_count = {"value": 0}  # Use mutable object to update inside function
model = YOLO('yolov8s.pt')
def process_frame(frame: np.ndarray, _) -> np.ndarray:
    results = model(frame, imgsz=1280)[0]
    detections = sv.Detections.from_ultralytics(results)
    zone.trigger(detections=detections)

    last_count["value"] = zone.current_count  # Update the latest count

    frame = box_annotator.annotate(scene=frame, detections=detections)
    frame = zone_annotator.annotate(scene=frame)
    return frame

# Run processing
start_time = time.time()
sv.process_video(source_path=video_path, target_path=output_path, callback=process_frame)

# Display final count

end_time = time.time()
elapsed_time = end_time - start_time
display.clear_output()
print(f"Time taken to process video: {elapsed_time:.2f} seconds")

count = last_count["value"]
if count < 10:
    wait = 20
elif 10 <= count < 40:
    wait = 40
else:  # count >= 40
    wait = 60

print(f"Count: {count}, Wait: {wait}")

Time taken to process video: 4.01 seconds
Count: 35, Wait: 40
